In [1]:
from myimports import (
    sys, tk, ttk, filedialog, mysql, pd,
    create_engine, text, Integer, Text, String, DateTime, Boolean,
    sns, plt, Image, ImageTk,
    large_font, medium_font, small_bold
)

from trainingmanager import TrainingManager
from tabletoolbar import TableToolbar
from dataframetoolbar import DataFrameToolbar

        
# Terminate the Tkinter event loop when the window is closed
def on_closing(*args):
    root.destroy()  
    exit() # Quit the Tkinter event loop and exit Python    

    
current_dataframe = pd.DataFrame()

# Create the main window
root = tk.Tk()
root.protocol("WM_DELETE_WINDOW", on_closing)  # Call on_closing() when the window is closed

exit_flag = tk.BooleanVar()

# Set the initial value of the boolean variable
exit_flag.set(False)

# Trace the changes to the boolean variable and bind the callback function
exit_flag.trace("w", on_closing)

root.title("Machine Learning Application Prototype")

root.maxsize(1200, 700)  # specify the max size the window can expand to
root.config(bg="skyblue")  # specify background color

# Create left and right frames
left_frame = tk.Frame(root, width=200, height=700, bg='grey')
left_frame.grid(row=0, column=0, padx=10, pady=10)


right_frame = tk.Frame(root, width=650, height=600, bg='grey')
right_frame.grid(row=0, column=1, padx=10, pady=10)

# Load the image
#image = Image.open("pairplot.jpg")
#photo = ImageTk.PhotoImage(image)

# Create a Canvas widget
canvas = tk.Canvas(right_frame, width=640, height=595)
canvas.grid(row=0, column=0, padx=5, pady=5, sticky='nsew')


# Display the initial image on the Canvas
image_item = canvas.create_image(0, 0, anchor='nw')

# Create a Scrollbar widget
canvas_yscrollbar = tk.Scrollbar(right_frame, orient=tk.VERTICAL, command=canvas.yview)
canvas_yscrollbar.grid(row=0, column=1, sticky='ns')

# Create a Scrollbar widget
canvas_xscrollbar = tk.Scrollbar(right_frame, orient=tk.HORIZONTAL, command=canvas.xview)
canvas_xscrollbar.grid(row=1, column=0, sticky='we')

# Configure the Canvas to use the Scrollbar
canvas.config(yscrollcommand=canvas_yscrollbar.set, xscrollcommand=canvas_xscrollbar.set)

dataframe_toolbar = DataFrameToolbar(left_frame, current_dataframe)

training_manager = TrainingManager(left_frame, canvas, image_item, current_dataframe, dataframe_toolbar)

table_toolbar = TableToolbar(left_frame, training_manager, exit_flag)


# Run the main event loop
root.mainloop()




IndentationError: expected an indented block (dataframetoolbar.py, line 18)